## Importing libraries

In [1]:
import numpy as np
import random
from typing import List
import numba as nb
import copy

## Define function and terminals

In [2]:
class function:

    def __init__(self, function, name, arity):
        self.function = function
        self.arity = arity
        self.name = name
        
    def __repr__(self) -> str:
        return self.name        

    def __call__(self, *args):
        return self.function(*args)

def _protected_division(x1, x2):
    """Closure of division (x1/x2) for zero denominator."""
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(np.abs(x2) > 0.001, np.divide(x1, x2), 1.)
    
add2 = function(function=np.add, name = "add", arity=2)
sub2 = function(function=np.subtract, name = "sub",  arity=2)
mul2 = function(function=np.multiply, name = "mul", arity=2)
max2 = function(function=np.maximum, name = "sub",  arity=2)
min2 = function(function=np.minimum, name = "mul", arity=2)
div2 = function(function=_protected_division, name = "div", arity=2)

## Genetic Programming

### Generate individual

In [3]:
def generate_individual(max_depth: int, 
                        function_set: List[function]):
    
    function = np.random.choice(function_set)
    terminal_stack = [function.arity]
    program = [function]
    while terminal_stack:
        depth = len(terminal_stack)
        
        if depth < max_depth:
            next_func = np.random.choice(function_set)
            program.append(next_func)
            terminal_stack.append(next_func.arity)
            
        else:
            program.append("X")
            terminal_stack[-1] -= 1
            while terminal_stack[-1] == 0:
                terminal_stack.pop()
                if not terminal_stack:
                    return program
                terminal_stack[-1] -= 1
    return program

### Construct fitness function

In [4]:
def target_function(x, a = 1, b = 100):
    return (a - x)**2 + b * (50 - x**2)**2

def evaluate_fitness(program, x: np.ndarray):
    x = np.array(x)
    stack = []
    if len(program) == 1:
        return np.sqrt(sum((x - target_function(x)) ** 2).mean())
    
    for func in program:
        if isinstance(func, function):
            stack.append([func])
        else:
            stack[-1].append(x)
        # print(stack)
        while len(stack[-1]) == stack[-1][0].arity + 1:
            func = stack[-1][0]
            res = func(*stack[-1][1:])
            if len(stack) != 1:
                stack.pop()
                stack[-1].append(res)
            else:
                # print("A", stack, stack[-1][0].arity)
                # res = func(*stack[-1][1:])
                # print(res)
                return np.sqrt(sum((res - target_function(x)) ** 2).mean())
            
    return None

### Genetic Operator

In [5]:
def get_subtree(program):
    probs = np.array([0.9 if isinstance(node, function) else 0.1
                        for node in program])
    probs = np.cumsum(probs / probs.sum())
    start = np.searchsorted(probs, np.random.uniform())
    
    stack = 1
    end = start
    
    while stack > end - start:
        node = program[end]
        if isinstance(node, function):
            stack += node.arity
            
        end += 1
        
    return start, end

def crossover(pa, pb):
    pa = copy.deepcopy(pa)
    pb = copy.deepcopy(pb)
    start_a, end_a = get_subtree(pa)
    start_b, end_b = get_subtree(pb)
    
    return pa[:start_a] + pb[start_b: end_b] + pa[end_a:], pb[:start_b] + pa[start_a: end_a] + pb[end_b:]


def subtree_mutation(individual, function_set, max_depth=2):
    mutated_individual = copy.deepcopy(individual)
    tmp = generate_individual(max_depth, function_set)
    crossover(mutated_individual, tmp)
    return mutated_individual    

### Main

In [6]:
def main(data, pop_size, generations, max_depth, function_set, crossover_rate = 0.8, mutation_rate = 0.15):
    population = [generate_individual(max_depth=max_depth, function_set=function_set) for _ in range(pop_size)]
    # print(population[0])
    population = sorted(population, key=lambda x: evaluate_fitness(x, data), reverse=False)
    
    for generation in range(generations):
        assert len(population) == pop_size
        offspring = []
        while len(offspring) < pop_size:
            pa, pb = np.random.randint(0, len(population) - 1, 2)
            pa, pb = population[pa], population[pb]
            rnd = random.random()
            if rnd < crossover_rate:
                off1, off2 = crossover(pa, pb)
            elif rnd < mutation_rate + crossover_rate:
                off1, off2 = subtree_mutation(pa, function_set, mutation_rate), subtree_mutation(pb, function_set, mutation_rate)
            else:
                off1, off2 = generate_individual(max_depth, function_set), generate_individual(max_depth, function_set)
            offspring.append(off1)
            offspring.append(off2)

        population.extend(offspring)
        population = sorted(population, key=lambda x: evaluate_fitness(x, data), reverse=False)
        population = population[:pop_size]
        
        best_individual = population[0]
        print(f"Generation {generation+1}: Best individual, Fitness - {evaluate_fitness(best_individual, data)}")
    return best_individual
data_points = np.array([i for i in range(-10, 11)])
best_ind = main(data_points, generations=100, pop_size=100, max_depth=8, function_set = [add2, sub2, mul2, div2, min2, max2])
        

Generation 1: Best individual, Fitness - 629798.9969023847
Generation 2: Best individual, Fitness - 629798.9969023847
Generation 3: Best individual, Fitness - 629798.9969023847
Generation 4: Best individual, Fitness - 629798.9969023847
Generation 5: Best individual, Fitness - 612981.759041205
Generation 6: Best individual, Fitness - 612935.70736649
Generation 7: Best individual, Fitness - 612908.748761943
Generation 8: Best individual, Fitness - 609298.3336112943
Generation 9: Best individual, Fitness - 609298.3336112943
Generation 10: Best individual, Fitness - 609298.3336112943
Generation 11: Best individual, Fitness - 607258.7327640912
Generation 12: Best individual, Fitness - 607258.7327640912
Generation 13: Best individual, Fitness - 607258.7327640912
Generation 14: Best individual, Fitness - 607258.4399306444
Generation 15: Best individual, Fitness - 606582.2876224214
Generation 16: Best individual, Fitness - 606582.2876224214
Generation 17: Best individual, Fitness - 606582.2876

C:\Users\Admin\AppData\Local\Temp\ipykernel_7468\3699534555.py:26: RuntimeWarning: overflow encountered in scalar add
  return np.sqrt(sum((res - target_function(x)) ** 2).mean())


Generation 20: Best individual, Fitness - 32616.347358341645
Generation 21: Best individual, Fitness - 32616.347358341645
Generation 22: Best individual, Fitness - 32616.347358341645
Generation 23: Best individual, Fitness - 27161.18176736793
Generation 24: Best individual, Fitness - 27161.18176736793
Generation 25: Best individual, Fitness - 27161.18176736793
Generation 26: Best individual, Fitness - 27161.18176736793
Generation 27: Best individual, Fitness - 23506.020569207372
Generation 28: Best individual, Fitness - 23506.020569207372


C:\Users\Admin\AppData\Local\Temp\ipykernel_7468\3699534555.py:8: RuntimeWarning: overflow encountered in scalar add
  return np.sqrt(sum((x - target_function(x)) ** 2).mean())


Generation 29: Best individual, Fitness - 23506.020569207372
Generation 30: Best individual, Fitness - 23506.020569207372
Generation 31: Best individual, Fitness - 23506.020569207372
Generation 32: Best individual, Fitness - 23506.020569207372
Generation 33: Best individual, Fitness - 23506.020569207372
Generation 34: Best individual, Fitness - 8031.355365067592
Generation 35: Best individual, Fitness - 8031.355365067592
Generation 36: Best individual, Fitness - 8031.355365067592


C:\Users\Admin\AppData\Local\Temp\ipykernel_7468\3699534555.py:26: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(sum((res - target_function(x)) ** 2).mean())


Generation 37: Best individual, Fitness - 7041.202170084311
Generation 38: Best individual, Fitness - 7041.202170084311
Generation 39: Best individual, Fitness - 6499.1621767732495
Generation 40: Best individual, Fitness - 6499.1621767732495
Generation 41: Best individual, Fitness - 6499.1621767732495
Generation 42: Best individual, Fitness - 6499.1621767732495
Generation 43: Best individual, Fitness - 6499.1621767732495
Generation 44: Best individual, Fitness - 6499.1621767732495
Generation 45: Best individual, Fitness - 6499.1621767732495
Generation 46: Best individual, Fitness - 6499.1621767732495
Generation 47: Best individual, Fitness - 5577.083825800003
Generation 48: Best individual, Fitness - 5577.083825800003
Generation 49: Best individual, Fitness - 5577.083825800003
Generation 50: Best individual, Fitness - 5577.083825800003
Generation 51: Best individual, Fitness - 5577.083825800003
Generation 52: Best individual, Fitness - 5577.083825800003
Generation 53: Best individual, 

In [7]:
best_ind

[sub,
 mul,
 add,
 'X',
 'X',
 add,
 add,
 'X',
 'X',
 'X',
 mul,
 sub,
 mul,
 mul,
 mul,
 'X',
 'X',
 sub,
 'X',
 'X',
 mul,
 mul,
 mul,
 'X',
 'X',
 mul,
 'X',
 'X',
 sub,
 sub,
 'X',
 'X',
 add,
 'X',
 'X',
 mul,
 mul,
 mul,
 'X',
 'X',
 mul,
 'X',
 'X',
 sub,
 mul,
 'X',
 'X',
 mul,
 mul,
 sub,
 'X',
 'X',
 mul,
 'X',
 'X',
 sub,
 sub,
 'X',
 'X',
 add,
 'X',
 'X',
 sub,
 sub,
 'X',
 'X',
 add,
 'X',
 'X']

In [8]:
evaluate_fitness(best_ind, [1])

42539.31847126844